# Code-to-Code Comparison: IEA Task 26

### National Renewable Energy Laboratory

#### Rob Hammond

##### 3 November 2022

In [1]:
from copy import deepcopy
from time import perf_counter
from pprint import pprint

import yaml
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

from wombat.core import Simulation
from wombat.core.library import IEA_26, load_yaml

pd.options.display.float_format = '{:,.2f}'.format
pd.set_option("display.max_rows", 1000)
pd.set_option("display.max_columns", 1000)

In [2]:
tech_salary_annual = 100000
techs = 30
capacity = 400 * 1000  # 400MW -> kW
tech_salary_annual * techs / capacity

7.5

In [3]:
configs = [
    "requests",
    "requests_100pct_reduction",
    "one_mobilization",
    "one_mobilization_100pct_reduction",
    "two_mobilizations",
    "two_mobilizations_100pct_reduction",
    "three_mobilizations",
    "three_mobilizations_100pct_reduction",
]
columns = deepcopy(configs)
results = {
    "availability - time based": [],
    "availability - production based": [],
    "capacity factor - net": [],
    "capacity factor - gross": [],
    "power production": [],
    "task completion rate": [],
    "total annual costs": [],
    "technicians": [],
    "materials": [],
    "vessels": [],
    "ctv cost": [],
    "hlv cost": [],
    "dsv cost": [],
    "cab cost": [],
    "manual reset": [],
    "minor repair": [],
    "major repair": [],
    "major replacement": [],
    "remote reset": [],
    "annual service": [],
    "bos": [],  #  substructure inspection + scour repair + substation inspection + small/large transformer repairs
    "total downtime": [],
    "ctv utilization": [],
    "hlv utilization": [],
    "dsv utilization": [],
    "cab utilization": [],
}

In [4]:
for config in configs:
    # Run the simulation
    start = perf_counter()
    sim = Simulation(IEA_26, f"{config}.yaml")
    sim.run()
    end = perf_counter()
    print(f"{sim.config.name.rjust(45)} | {(end - start) / 60:.2f} m")
    
    # Gather the results of interest
    years = sim.metrics.events.year.unique().shape[0]
    mil = 1000000
    
    availability = sim.metrics.time_based_availability(frequency="project", by="windfarm").values[0][0]
    availability_production = sim.metrics.production_based_availability(frequency="project", by="windfarm").values[0][0]
    cf_net = sim.metrics.capacity_factor(which="net", frequency="project", by="windfarm").values[0][0]
    cf_gross = sim.metrics.capacity_factor(which="gross", frequency="project", by="windfarm").values[0][0]
    power_production = sim.metrics.power_production(frequency="project", by_turbine=False).values[0][0]
    completion_rate = sim.metrics.task_completion_rate(which="both", frequency="project").values[0][0]
    parts = sim.metrics.events[["materials_cost"]].sum().sum()
    techs = sim.metrics.project_fixed_costs(frequency="project", resolution="medium").labor[0]
    total = sim.metrics.events[["total_cost"]].sum().sum()

    equipment = sim.metrics.equipment_costs(frequency="project", by_equipment=True)
    equipment_sum = equipment.sum().sum()
    ctv = equipment[[el for el in equipment.columns if "Crew Transfer Vessel" in el]].sum().sum()
    hlv = equipment[[el for el in equipment.columns if "Jack-up Vessel" in el]].sum().sum()
    dsv = equipment[[el for el in equipment.columns if "Diving Support Vessel" in el]].sum().sum()
    cab = equipment[[el for el in equipment.columns if "Cable Laying Vessel" in el]].sum().sum()

    times = sim.metrics.process_times()
    times = times / years / 24 / 100  # events per turbine and year
    
    utilization = sim.metrics.service_equipment_utilization(frequency="project")
    ctv_ur = utilization[[el for el in utilization.columns if "Crew Transfer Vessel" in el]].mean().mean()
    hlv_ur = utilization[[el for el in utilization.columns if "Jack-up Vessel" in el]].mean().mean()
    dsv_ur = utilization[[el for el in utilization.columns if "Diving Support Vessel" in el]].mean().mean()
    cab_ur = utilization[[el for el in utilization.columns if "Cable Laying Vessel" in el]].mean().mean()

    # Log the results of interest
    results["availability - time based"].append(availability)
    results["availability - production based"].append(availability_production)
    results["capacity factor - net"].append(cf_net)
    results["capacity factor - gross"].append(cf_gross)
    results["power production"].append(power_production)
    results["task completion rate"].append(completion_rate)
    results["total annual costs"].append((total + techs) / mil / years)
    results["technicians"].append(techs / mil / years)
    results["materials"].append(parts / mil / years)
    results["vessels"].append(equipment_sum / mil / years)
    results["ctv cost"].append(ctv / mil / years)
    results["hlv cost"].append(hlv / mil / years)
    results["dsv cost"].append(dsv / mil / years)
    results["cab cost"].append(cab / mil / years)
    results["manual reset"].append(times.loc[times.index.intersection(["manual reset"]), "downtime"].sum())
    results["minor repair"].append(times.loc[times.index.intersection(["minor repair", ]), "downtime"].sum())
    results["major repair"].append(times.loc[times.index.intersection(["major repair"]), "downtime"].sum())
    results["major replacement"].append(times.loc[times.index.intersection(["major replacement"]), "downtime"].sum())
    results["remote reset"].append(times.loc[times.index.intersection(["remote reset"]), "downtime"].sum())
    results["annual service"].append(times.loc[times.index.intersection(["annual service"]), "downtime"].sum())
    ix = [
        "substructure inspection", "substation inspection",
        "small foundation/scour repair", "cable replacement",
        "small transformer repair", "large transformer repair"
    ]
    results["bos"].append(times.loc[times.index.intersection(ix), "downtime"].sum())
    results["total downtime"].append(times.loc[:, "downtime"].sum())
    results["ctv utilization"].append(ctv_ur)
    results["hlv utilization"].append(hlv_ur)
    results["dsv utilization"].append(dsv_ur)
    results["cab utilization"].append(cab_ur)
    sim.env.cleanup_log_files(log_only=True)

                              iea_26_requests | 3.58 m
             iea_26_requests_100pct_reduction | 2.04 m
                      iea_26_one_mobilization | 3.14 m
     iea_26_one_mobilization_100pct_reduction | 2.07 m
                     iea_26_two_mobilizations | 3.15 m
    iea_26_two_mobilizations_100pct_reduction | 2.06 m
                   iea_26_three_mobilizations | 3.25 m
  iea_26_three_mobilizations_100pct_reduction | 2.07 m


In [5]:
# Save the results
# pickled dictionary format
with open(IEA_26/ "outputs" / "results_dict_v0.5.2.yaml", "w") as f:
    yaml.dump(results, f, default_flow_style=False, sort_keys=False)

# dataframe/csv format
results_df = pd.DataFrame(results.values(), columns=columns, index=results.keys()).fillna(0)
results_df.to_csv(IEA_26 / "outputs" / "results_data_v0.5.2.csv", index_label="result")

In [6]:
results_df

,requests,requests_100pct_reduction,one_mobilization,one_mobilization_100pct_reduction,two_mobilizations,two_mobilizations_100pct_reduction,three_mobilizations,three_mobilizations_100pct_reduction
availability - time based,0.96,0.98,0.89,0.99,0.97,0.99,0.97,0.99
availability - production based,0.97,0.98,0.90,0.99,0.97,0.99,0.98,0.99
capacity factor - net,0.57,0.57,0.53,0.58,0.57,0.58,0.57,0.58
capacity factor - gross,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59
power production,"39,964,728,387.50","40,236,346,515.50","36,892,903,674.50","40,618,186,828.00","39,981,858,421.00","40,581,683,426.50","40,200,083,708.00","40,676,501,514.50"
task completion rate,0.99,0.97,1.00,1.00,1.00,0.99,0.99,1.00
total annual costs,14.90,6.24,14.33,11.15,17.51,14.43,20.95,17.04
technicians,3.00,3.00,3.00,3.15,3.00,3.15,3.15,3.15
materials,3.31,0.07,3.21,0.09,3.24,0.06,3.26,0.07
vessels,8.59,3.18,8.12,7.90,11.27,11.21,14.54,13.81
